In [1]:
# In the output, check that you have Tesla V100 (best), Tesla P100, or something similar

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed May  5 09:02:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
dtype = 'float32'
tf.keras.backend.set_floatx(dtype)

In [ ]:
# fashion_mnist = tf.keras.datasets.fashion_mnist
# (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# X_train = X_train.astype(dtype) / 255.0
# y_train = y_train.astype(dtype)
# X_test = X_test.astype(dtype)  / 255.0
# y_test = y_test.astype(dtype)

# X_train = np.reshape(X_train, (-1, 784))
# X_test = np.reshape(X_test, (-1, 784))

In [4]:
cifar10 = tf.keras.datasets.cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train.astype(dtype) / 255.0
y_train = y_train.astype(dtype)
X_test = X_test.astype(dtype)  / 255.0
y_test = y_test.astype(dtype)

X_train = np.reshape(X_train, (-1, 3072))
X_test = np.reshape(X_test, (-1, 3072))

170500096/170498071 [==============================] - 6s 0us/step


In [32]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [47]:
class SSRegularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, l1):
        self.l1 = l1

    def __call__(self, x):
        scaling_vector = tf.cumsum(tf.constant(self.l1, shape=(x.shape[0],), dtype=dtype), axis=0) - self.l1
        return tf.reduce_sum(tf.reshape(scaling_vector, (-1, 1)) * tf.abs(x))

    def get_config(self):
        return {'l1': float(self.l1)}


class SSLayer(tf.keras.Model):
    def __init__(self, input_units, units, activation, l1, kernel_initializer, bias_initializer, regularize=True):
        super().__init__()

        self.input_units = input_units
        self.units = units
        self.activation = activation
        self.l1 = l1
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        
        self.A = tf.keras.activations.get(activation)
        self.W_init = tf.keras.initializers.get(kernel_initializer)
        self.b_init = tf.keras.initializers.get(bias_initializer)
        self.regularizer = SSRegularizer(self.l1)
        
        self.W = tf.Variable(
            name='W',
            initial_value=self.W_init(shape=(input_units, units), dtype=dtype),
            trainable=True)
        
        self.b = tf.Variable(
            name='b',
            initial_value=self.b_init(shape=(units,), dtype=dtype),
            trainable=True)
        
        if regularize:
            self.add_loss(lambda: self.regularizer(self.W))
            self.add_loss(lambda: self.regularizer(self.b))
    
    def call(self, inputs):
        return self.A(tf.matmul(inputs, self.W) + self.b)
    
    def copy_without_regularization(self):
        copy = SSLayer(self.input_units, self.units, self.activation, self.l1, self.kernel_initializer, self.bias_initializer, regularize=False)
        copy.W = self.W
        copy.b = self.b
        return copy


class SSModel(tf.keras.Model):
    def __init__(self, layer_sizes, activation=None, l1=0.01, kernel_initializer='glorot_uniform', bias_initializer='zeros'):
        super().__init__()
        
        self.sslayers = list()
        for l in range(len(layer_sizes) - 1):
            input_units = layer_sizes[l]
            units = layer_sizes[l + 1]
            if l == 0:  # First layer
                layer = SSLayer(input_units, units, activation, 0., kernel_initializer, bias_initializer)
            elif l == len(layer_sizes) - 2:  # Last layer
                layer = SSLayer(input_units, units, 'softmax', l1, kernel_initializer, bias_initializer)
            else:  
                layer = SSLayer(input_units, units, activation, l1, kernel_initializer, bias_initializer)
            self.sslayers.append(layer)

    def call(self, inputs):
        x = inputs
        for layer in self.sslayers:
            x = layer(x)
        return x
    
    def get_layer_sizes(self):
        layer_sizes = list()
        for l in range(len(self.sslayers)):
            layer = self.sslayers[l]
            layer_sizes.append(layer.W.shape[0])
            if l == len(self.sslayers) - 1:  # Last layer
                layer_sizes.append(layer.W.shape[1])
        return layer_sizes
    
    def print_neurons(self):
        for layer in self.sslayers[1:]:
            print(get_param_string(layer.W))
    
    def remove_regularization(self):
        for l in range(len(self.sslayers)):
            self.sslayers[l] = self.sslayers[l].copy_without_regularization()
        
    def prune(self, threshold=0.001):
        for l in range(len(self.sslayers) - 1):
            layer1 = self.sslayers[l]
            layer2 = self.sslayers[l + 1]
            
            W1 = layer1.W.value()
            b1 = layer1.b.value()
            W2 = layer2.W.value()

            neurons_are_active = tf.math.reduce_max(tf.abs(W2), axis=1) >= threshold
            active_neurons_indices = tf.reshape(tf.where(neurons_are_active), (-1,))
            
            new_W1 = tf.gather(W1, active_neurons_indices, axis=1)
            new_b1 = tf.gather(b1, active_neurons_indices, axis=0)
            new_W2 = tf.gather(W2, active_neurons_indices, axis=0)

            layer1.W = tf.Variable(name='W', initial_value=new_W1, trainable=True)
            layer1.b = tf.Variable(name='b', initial_value=new_b1, trainable=True)
            layer2.W = tf.Variable(name='W', initial_value=new_W2, trainable=True)
    
    def grow(self, min_new_neurons=5, scaling_factor=0.001):   
        for l in range(len(self.sslayers) - 1):
            layer1 = self.sslayers[l]
            layer2 = self.sslayers[l + 1]
       
            W1 = layer1.W.value()
            b1 = layer1.b.value()
            W2 = layer2.W.value()

            n_new_neurons = max(min_new_neurons, int(W1.shape[1] * 0.2))

            W1_growth = layer1.W_init(shape=(W1.shape[0], W1.shape[1] + n_new_neurons), dtype=dtype)[:, -n_new_neurons:] * scaling_factor
            b1_growth = layer1.b_init(shape=(n_new_neurons,), dtype=dtype)
            W2_growth = layer2.W_init(shape=(W2.shape[0] + n_new_neurons, W2.shape[1]), dtype=dtype)[-n_new_neurons:, :]

            new_W1 = tf.concat([W1, W1_growth], axis=1)
            new_b1 = tf.concat([b1, b1_growth], axis=0)
            new_W2 = tf.concat([W2, W2_growth], axis=0)

            layer1.W = tf.Variable(name='W1', initial_value=new_W1, trainable=True)
            layer1.b = tf.Variable(name='b1', initial_value=new_b1, trainable=True)
            layer2.W = tf.Variable(name='W2', initial_value=new_W2, trainable=True)

In [49]:
def get_param_string(weights):
    param_string = ""
    max_parameters = tf.math.reduce_max(tf.abs(weights), axis=1).numpy()
    magnitudes = np.floor(np.log10(max_parameters))
    for m in magnitudes:
        if m > 0:
            m = 0
        param_string += str(int(-m))
    return param_string


def print_epoch_statistics(model, x, y, validation_data):
    x_val = validation_data[0]
    y_val = validation_data[1]

    y_pred = model(x)
    loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y, y_pred))
    accuracy = tf.reduce_mean(tf.keras.metrics.sparse_categorical_accuracy(y, y_pred))
    
    y_val_pred = model(x_val)
    val_loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_val, y_val_pred))
    val_accuracy = tf.reduce_mean(tf.keras.metrics.sparse_categorical_accuracy(y_val, y_val_pred))
    print(f"loss: {loss} - accuracy: {accuracy} - val_loss: {val_loss} - val_accuracy: {val_accuracy}")
    print(f"layer sizes: {model.get_layer_sizes()}")
    model.print_neurons()


def train_model(model, x, y, optimizer, epochs, self_scaling_epochs, batch_size, validation_data):
    train_dataset = tf.data.Dataset.from_tensor_slices((x, y))
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")

        if epoch < self_scaling_epochs:
            print("Before growing:")
            print_epoch_statistics(model, x, y, validation_data)
            model.grow(min_new_neurons=100, scaling_factor=0.001)
            print("After growing:")
            print_epoch_statistics(model, x, y, validation_data)
        
        if epoch == self_scaling_epochs:
            model.remove_regularization()

        for step, (x_batch, y_batch) in enumerate(train_dataset):
            with tf.GradientTape() as tape:
                y_pred = model(x_batch, training=True)
                loss_value = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_batch, y_pred))
                loss_value += sum(model.losses)

            grads = tape.gradient(loss_value, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        if epoch < self_scaling_epochs:
            print("Before pruning:")
            print_epoch_statistics(model, x, y, validation_data)
            model.prune(threshold=0.001)
            print("After pruning:")
            print_epoch_statistics(model, x, y, validation_data)
        else:
            print_epoch_statistics(model, x, y, validation_data)

In [66]:
epochs = 50
self_scaling_epochs = 10
batch_size = 32

model = SSModel(layer_sizes=[3072, 1000, 1000, 1000, 1000, 1000, 1000, 10], activation='selu', l1=0.00001, kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, validation_data=(X_test_norm, y_test))

Epoch 1/50
Before growing:
loss: 2.757193088531494 - accuracy: 0.09390000253915787 - val_loss: 2.7507741451263428 - val_accuracy: 0.09139999747276306
layer sizes: [3072, 1000, 1000, 1000, 1000, 1000, 1000, 10]
2222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222

In [71]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1639, activation='selu', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(353, activation='selu', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(319, activation='selu', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(268, activation='selu', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(191, activation='selu', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(190, activation='selu', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(10, activation='softmax', kernel_initializer='he_normal'),
])

In [72]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

In [73]:
model.fit(X_train_norm, y_train, epochs=50, validation_data=(X_test_norm, y_test))

Epoch 1/50
1563/1563 [==============================] - 7s 4ms/step - loss: 3.1681 - accuracy: 0.2944 - val_loss: 1.7189 - val_accuracy: 0.3943
Epoch 2/50
1563/1563 [==============================] - 6s 4ms/step - loss: 1.5894 - accuracy: 0.4342 - val_loss: 1.5098 - val_accuracy: 0.4641
Epoch 3/50
1563/1563 [==============================] - 6s 4ms/step - loss: 1.4834 - accuracy: 0.4753 - val_loss: 1.5138 - val_accuracy: 0.4493
Epoch 4/50
1563/1563 [==============================] - 6s 4ms/step - loss: 1.4093 - accuracy: 0.4959 - val_loss: 1.4429 - val_accuracy: 0.4871
Epoch 5/50
1563/1563 [==============================] - 6s 4ms/step - loss: 1.3612 - accuracy: 0.5194 - val_loss: 1.5563 - val_accuracy: 0.4531
Epoch 6/50
1563/1563 [==============================] - 6s 4ms/step - loss: 1.2742 - accuracy: 0.5489 - val_loss: 1.4225 - val_accuracy: 0.5057
Epoch 7/50
1563/1563 [==============================] - 6s 4ms/step - loss: 1.2158 - accuracy: 0.5689 - val_loss: 1.4178 - val_accuracy: